## Introduction
#### The ***2025* Formula 1 season** represented the **final and most saturated year of the ground-effect regulation cycle**, with teams operating at the limits of aerodynamic and operational convergence ahead of the 2026 reset; while **Red Bull and Max Verstappen** continued to be central reference points, the season was defined by **compressed performance margins**, increased strategic sensitivity, and heightened importance of execution over outright pace, as Ferrari, McLaren, and Mercedes capitalised on marginal gains, tyre management, and race-day decisions, making 2025 a season characterised by **regulation exhaustion**, **fine-grained strategic trade-offs**, and a grid finely balanced on the edge of a major technical transition.


## Importing the necessary libraries

In [2]:
import pandas as pd
import fastf1
import logging

In [3]:
fastf1.Cache.enable_cache("../cache")

## Getting the Races of 2025

In [4]:
schedule = fastf1.get_event_schedule(2025)
listOfRaces = schedule["Country"].tolist()
listOfRaces

['Bahrain',
 'Australia',
 'China',
 'Japan',
 'Bahrain',
 'Saudi Arabia',
 'United States',
 'Italy',
 'Monaco',
 'Spain',
 'Canada',
 'Austria',
 'United Kingdom',
 'Belgium',
 'Hungary',
 'Netherlands',
 'Italy',
 'Azerbaijan',
 'Singapore',
 'United States',
 'Mexico',
 'Brazil',
 'United States',
 'Qatar',
 'United Arab Emirates']

## Iterating into every Race and concatenating every Race Data into a DataFrame
Through my first try in ["Bahrain_test.ipynb"](https://github.com/Chracker24/MTS-IE/blob/main/02_Notebooks/Data_Collection/Formula1/Pre-Ground%20Effect%20Era/2020/Bahrain_test.ipynb), I am going to use a loop to collect data and trim it down to the column and data points that I deem necessary for the Intelligence Engine.

In [5]:
cols = [
    "Driver",
    "LapTime",
    "LapNumber",
    "Stint",
    "Sector1Time",
    "Sector2Time",
    "Sector3Time",
    "Compound",
    "Team",
    "Deleted",
]

In [6]:
Season_Data2025 = []

In [7]:
logging.getLogger("fastf1").setLevel(logging.ERROR)
for i in range(len(listOfRaces)):
    session = fastf1.get_session(2025, listOfRaces[i], "R")
    session.load(laps=True, telemetry=False, weather=False)
    laps = session.laps
    Race_Data = laps[cols].copy()
    Race_Data = Race_Data.dropna(subset=["LapTime"]).copy()
    Race_Data["Season"] = 2025
    Race_Data["Race"] = listOfRaces[i]
    Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]] = Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]].apply(lambda x : x.dt.total_seconds())
    Race_Data = Race_Data[Race_Data["Deleted"]==False].copy()

    Season_Data2025.append(Race_Data)

In [8]:
season_2025 = pd.concat(Season_Data2025, ignore_index=True)

In [11]:
season_2025 = season_2025.sort_values("LapNumber").reset_index(drop=True)
season_2025

,Driver,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,Compound,Team,Deleted,Season,Race
0,BEA,94.986,1.0,1.0,NaN,31.363,26.086,MEDIUM,Haas F1 Team,False,2025,Qatar
1,RUS,134.241,1.0,1.0,NaN,51.124,38.639,HARD,Mercedes,False,2025,Azerbaijan
2,ALB,92.851,1.0,1.0,NaN,31.206,29.271,HARD,Williams,False,2025,Italy
3,COL,96.577,1.0,1.0,NaN,41.290,22.860,HARD,Alpine,False,2025,Monaco
4,ALB,84.528,1.0,1.0,NaN,28.307,23.575,MEDIUM,Williams,False,2025,Netherlands
...,...,...,...,...,...,...,...,...,...,...,...,...
27056,LEC,75.369,78.0,3.0,19.466,35.723,20.180,MEDIUM,Ferrari,False,2025,Monaco
27057,NOR,73.221,78.0,3.0,19.145,34.697,19.379,HARD,McLaren,False,2025,Monaco
27058,VER,77.770,78.0,3.0,22.061,35.599,20.110,SOFT,Red Bull Racing,False,2025,Monaco
27059,HAM,77.439,78.0,3.0,20.498,36.035,20.906,MEDIUM,Ferrari,False,2025,Monaco


## Adding the Dataframe as a CSV file in Data/Archive

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[3]
archive_dir = PROJECT_ROOT / "Data" / "Archive"

archive_dir.mkdir(parents=True, exist_ok=True)

season_2025.to_csv(
    archive_dir / "season_2025.csv",
    index=False
)